<a href="https://colab.research.google.com/github/dkalenov/ML-Stock-Crypto-Trading/blob/4_trading/Binance_Futures_Test_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run

Введи свой Api

In [ ]:
api = "YOUR API"

api_secret = 'YOUR SECRET API'

In [ ]:
!pip install python-binance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 679.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.4 MB/s eta 0:00:00


## Bot running function

In [ ]:
from binance.client import Client
import pandas as pd
import requests


client = Client(api,api_secret,tld="com",testnet=True)

class Bot:
    def __init__(self,symbol,no_of_decimals,volume,proportion,tp,n):
        self.symbol = symbol
        self.no_of_decimals = no_of_decimals
        self.volume = volume
        self.proportion = proportion
        self.tp = tp
        self.n = n

    def get_balance(self):
        x = client.futures_account()
        df = pd.DataFrame(x['assets'])
        display(df)

    def get_current_price(self,symbol):
        response = requests.get(f'https://testnet.binancefuture.com/fapi/v1/ticker/price?symbol={symbol}')
        price = float(response.json()['price'])
        return price

    def sell_limit(self,symbol, volume, price):
        output = client.futures_create_order(

            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.FUTURE_ORDER_TYPE_LIMIT,
            timeInForce=Client.TIME_IN_FORCE_GTC,
            quantity=volume,
            price=price,
        )
        print(output)

    def buy_limit(self,symbol, volume, price):
        output = client.futures_create_order(

            symbol=symbol,
            side=Client.SIDE_BUY,
            type=Client.FUTURE_ORDER_TYPE_LIMIT,
            timeInForce=Client.TIME_IN_FORCE_GTC,
            quantity=volume,
            price=price,
        )
        print(output)

    def close_orders(self,symbol):
        x = client.futures_get_open_orders(symbol=symbol)
        df = pd.DataFrame(x)
        for index in df.index:
            client.futures_cancel_order(symbol=symbol, orderId=df["orderId"][index])

    def close_buy_orders(self,symbol):
        x = client.futures_get_open_orders(symbol=symbol)
        df = pd.DataFrame(x)
        df = df[df["side"] == "BUY"]
        for index in df.index:
            client.futures_cancel_order(symbol=symbol, orderId=df["orderId"][index])

    def close_sell_orders(self,symbol):
        x = client.futures_get_open_orders(symbol=symbol)
        df = pd.DataFrame(x)
        df = df[df["side"] == "SELL"]
        for index in df.index:
            client.futures_cancel_order(symbol=symbol, orderId=df["orderId"][index])

    def get_direction(self,symbol):
        x = client.futures_position_information(symbol=symbol)
        df = pd.DataFrame(x)
        if float(df["positionAmt"].sum()) > 0:
            return "LONG"
        if float(df["positionAmt"].sum()) < 0:
            return "SHORT"
        else:
            return "FLAT"

    def get_mark_price(self,symbol):
        x = client.get_symbol_ticker(symbol=symbol)
        price = float(x["price"])
        return price

    def draw_grid(self,n):
        pct_change = 1
        adj_sell = 1.2
        current_price = self.get_mark_price(self.symbol)
        for i in range(n):
            sell_price = float(
                round(((pct_change / 100) * current_price * adj_sell * self.proportion) + current_price, self.no_of_decimals))
            self.sell_limit(self.symbol, self.volume, sell_price)
            pct_change += 1
            adj_sell += 0.2

        pct_change = -1
        adj_buy = 1.2
        current_price = self.get_mark_price(self.symbol)
        for i in range(n):
            buy_price = float(
                round(((pct_change / 100) * current_price * adj_sell * self.proportion) + current_price, self.no_of_decimals))
            self.buy_limit(self.symbol, self.volume, buy_price)
            pct_change -= 1
            adj_buy += 0.2

    def cal_tp_level(self,symbol, tp):
        try:
            x = client.futures_position_information(symbol=symbol)
            df = pd.DataFrame(x)
            df = df.loc[df["positionAmt"] != "0.000"]
            t_margin = (float(df["entryPrice"][0]) * abs(float(df["positionAmt"][0]))) / float(df["leverage"][0])
            profit = float(t_margin * tp * 0.01)
            price = round((profit / float(df["positionAmt"][0])) + float(df["entryPrice"][0]), self.no_of_decimals)
            t_position_amt = 0
            for index in df.index:
                t_position_amt += abs(float(df["positionAmt"][index]))
            return price, t_position_amt

        except:
            pass

    def place_tp_order(self,symbol, price, t_position_amt, direction):
        try:
            if direction == "LONG":
                self.sell_limit(symbol, t_position_amt, price)
            if direction == "SHORT":
                self.buy_limit(symbol, t_position_amt, price)
        except:
            self.place_tp_order(symbol, price, t_position_amt, direction)


    def run(self):
        while True:
            x = client.futures_get_open_orders(symbol=self.symbol)
            df1 = pd.DataFrame(x)
            if len(df1) == 0:
                self.draw_grid(self.n)
            y = client.futures_position_information(symbol=self.symbol)
            df2 = pd.DataFrame(y)
            df2 = df2.loc[df2["positionAmt"] != "0.000"]
            if len(df2) > 0:
                direction = self.get_direction(self.symbol)
                try:
                    if direction == "LONG":
                        print("close buy")
                        self.close_sell_orders(self.symbol)
                    if direction == "SHORT":
                        print("close sell")
                        self.close_buy_orders(self.symbol)
                except:
                    pass
                price0, amount0 = self.cal_tp_level(self.symbol, self.tp)
                self.place_tp_order(self.symbol, price0, amount0, direction)
                is_ok = True
                while is_ok:
                    try:
                        price1, amount1 = self.cal_tp_level(self.symbol, self.tp)
                        print(f"price: {price1} amount: {amount1}")
                        if price1 != price0 or amount1 != amount0:
                            if direction == "LONG":
                                self.close_sell_orders(self.symbol)
                            if direction == "SHORT":
                                self.close_buy_orders(self.symbol)
                            self.place_tp_order(self.symbol, price1, amount1, direction)
                            price0 = price1
                            amount0 = amount1
                    except:
                        pass

                    y = client.futures_position_information(symbol=self.symbol)
                    df2 = pd.DataFrame(y)
                    df2 = df2.loc[df2["positionAmt"] != "0.000"]
                    if len(df2) == 0:
                        try:
                            self.close_orders(self.symbol)
                            is_ok = False
                        except:
                            pass

## Bot runnig

Bot parameters 'Bot("BTCUSDT", 1, 0.01, 0.04, 5, 10)' or 'Bot(symbol, no_of_decimals, volume, proportion, tp, n)':

1. symbol: symbol of the trading pair, for example, "BTCUSDT" or "ETHUSDT"
2. no_of_decimals: number of decimal places for price rounding
3. volume: transaction volume
4. proportion: proportion of price changes when placing orders
5. tp: target profit (take profit) in percentage
6. n: number of orders placed on each side



In [ ]:
from threading import Thread

symbol = "BTCUSDT" # trading pair symbol
no_of_decimals = 1 # number of decimal places for price rounding
volume = 0.05 # trade volume
proportion = 0.03 # proportion of price changes when placing orders
tp = 5 # target profit (take profit) in percentage
n = 10 # number of orders placed on each side of the grid

symbol2 = "ETHUSDT"
no_of_decimals2 = 1
volume2 = 0.09
proportion2 = 0.03
tp2 = 5
n2 = 10

bot1 = Bot(symbol, no_of_decimals, volume, proportion, tp, n)
bot2 = Bot(symbol2, no_of_decimals2, volume2, proportion2, tp2, n2)

def b1():
    bot1.run()
def b2():
    bot2.run()

t1 = Thread(target=b1)
t2 = Thread(target=b2)

t1.start()
t2.start()

In [ ]:
# Current balance

Bot.get_balance(bot1)

,asset,walletBalance,unrealizedProfit,marginBalance,maintMargin,initialMargin,positionInitialMargin,openOrderInitialMargin,maxWithdrawAmount,crossWalletBalance,crossUnPnl,availableBalance,marginAvailable,updateTime
0,FDUSD,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
1,BTC,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
2,BNB,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
3,ETH,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
4,USDT,24610.61104441,154.74948573,24765.36053014,125.64070565,1570.50882071,1570.50882071,0.00000000,23194.85170943,24610.61104441,154.74948573,23194.85170943,True,1721098859216
5,USDC,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0


## Operating principle

How the bot works:
1. The loop checks whether there are open buy or sell orders on the market. If not, then a new order grid is created.
2. Then information about current market positions is obtained. If there are open positions, the direction (LONG or SHORT) is determined and appropriate actions are taken to close orders in the opposite direction.
3. Prices and volumes are calculated to set take-profit levels. Sell ​​or buy orders are then created depending on the direction of the position.
4. The cycle checks whether prices or volumes for take-profit levels have changed. If yes, then current orders are closed and new ones are created with updated values.
5. Periodically check whether there are still open positions on the market. If not, then all orders are closed and the cycle begins again.